# Cargar Datos de Agentes Libres
Este notebook se encarga actualizar la tabla con los datos de los Agentes Libres extraidos de Overthecap. Tan sólo se ejecuta cuando quieres refrescar los que han firmado/renovado o si se han borrado las tablas.

## Pre-procesado manual
Los datos se extraen manualmente de la web siguiendo este proceso:

- Ve a la web de Overthecap, selecciona toda la tabla y copia.
- Pega el contenido en Excel y guárdalo como CSV
- Desde Catalog, sube el fichero a `/Volumes/nfl-data/overthecap`

In [0]:
# File location and type
file_location = "dbfs:/Volumes/nfl-data/overthecap/2026/overthecap_freeagents_20260119.csv"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("mergeSchema", "true") \
  .option("sep", ",") \
  .load(file_location)

#Conversiones
En esta seccion se implementan las distintas conversiones de los datos.

In [0]:
from pyspark.sql.functions import translate, col, when
from pyspark.sql.types import IntegerType, FloatType

# Rename columns with unsupported names by Delta
df = df.withColumnRenamed("Pos.", "Pos") \
       .withColumnRenamed("2025 Team", "PrevTeam") \
       .withColumnRenamed("2026 Team", "NextTeam") \
       .withColumnRenamed("Snaps", "Snaps%") \
       .withColumnRenamed("Current APY", "SalaryPY")

# Convert Vvoid contracts to UFA
df = df.withColumn('Type', when(col('Type') == 'Void', 'UFA').otherwise(col('Type')))

# Convert string currency to int
df = df.withColumn('Guarantees', translate(col("Guarantees"), "$,", "").cast(IntegerType())) \
       .withColumn('SalaryPY', translate(col("SalaryPY"), "$,", "").cast(IntegerType())) \
       .withColumn('Snaps%', translate(col("Snaps%"), "%", "").cast(FloatType()).cast(IntegerType()))

display(df)
df.printSchema()

Corregir aquellos nombres de jugadores (PK) que se escriben diferente en PFF.


In [0]:
df = df.withColumn('Player', when(col('Player') == 'Cordale Flott', "Cor'Dale Flott").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'D.J. Reader', "DJ Reader").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Kenneth Murray', "Kenneth Murray Jr.").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Kris Boyd', "Khristian Boyd").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Chauncey Gardner-Johnson, Jr.', "C.J. Gardner-Johnson").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Deatrich Wise', "Deatrich Wise Jr.").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Cameron Lewis', "Cam Lewis").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Willie Gay, Jr.', "Willie Gay").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Ugochukwu Amadi', "Ugo Amadi").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Cameron Sample', "Cam Sample").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Ogbonnia Okoronkwo', "Ogbo Okoronkwo").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Antonio Hamilton', "Antonio Hamilton Sr.").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Darnell Savage Jr.', "Darnell Savage").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Sam Franklin', "Sam Franklin Jr.").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Rodney Thomas', "Rodney Thomas II").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'A.J. Green', "A.J. Green III").otherwise(col('Player')))
# Ataque
df = df.withColumn('Player', when(col('Player') == 'Trey Pipkins', "Trey Pipkins III").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Mitchell Trubisky', "Mitch Trubisky").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Joseph Noteboom', "Joe Noteboom").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Trenton Scott', "Trent Scott").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'James Proche', "James Proche II").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Gabriel Davis', "Gabe Davis").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Thayer Munford', "Thayer Munford Jr.").otherwise(col('Player')))
df = df.withColumn('Player', when(col('Player') == 'Yosuah Nijman', "Yosh Nijman").otherwise(col('Player')))

# DEF que no han jugado: Kader Kohou, Miles Killebrew, Dean Lowry, Payton Turner, Troy Andersen, Martin Emerson, Artie Burns, J.T. Gray, Deon Bush, George Odum, C.J. Goodwin, Zeke Turner, Jon Rhattigan, Caleb Johnson, Robert Rochell, Damarri Mathis, J.J. Russell, Brian Asamoah, Jacob Phillips, Janarius Robinson
# OFE que no han jugado Evan Neal, Sam Howell, Yodny Cajuste, Easton Stick, Ryan Bates, Robert Jones, Cordell Volson, Landon Young, Case Keenum, Liam Eichenberg, Rondale Moore, Will Clapp, DeAndre Carter, Michael Burton, Joshua Ezeudu, Chukwuma Okorafor, Germain Ifedi, Jeff Driskel, George Fant, John Wolford, Sam Ehlinger, Trent Taylor, Max Scharping, Donald Parham, Matt Nelson, Jack Driscoll, Skylar Thompson, Lucas Niang, Jamarco Jones

# Actualiza Tabla
Crea la Delta table con todos los datos y ciertas conversiones que me los hacen más manejables.

In [0]:
permanent_table_name = "freeagents_2026"

df.write.format("delta").saveAsTable(permanent_table_name, mode="overwrite")

# Troubleshooting and Testing

Usa estas herramientas y comandos para comprobar el estado del workspace


In [0]:
# Delete un-used files/folders
#dbutils.fs.rm("dbfs:/FileStore/tables/nfl/pff/defense_summary.csv", True)
# List files
dbutils.fs.ls("dbfs:/Workspace/")
dbutils.fs.mkdirs("dbfs:/Workspace/reports")
#dbutils.fs.rm("dbfs:/FileStore/tables/nfl/pff/defense_summary.csv"

Una prueba rápida para ver que funciona la Delta Table.


In [0]:
%sql

select * from freeagents_2026 where Pos == "EDGE";

# Troubleshooting
Se usa para comprobar entradas en los datos the OverTheCap que no están en los de PFF.
Esto puede ocurrir por dos razones:
- PFF usa un nombre diferente para el mismo jugador
- El jugador no ha disputado ningún snap en defensa/ataque

In [0]:
%sql
SELECT fa.Player, fa.Pos, fa.PrevTeam, fa.`Snaps%`
  FROM freeagents_2026 as fa 
--  FULL OUTER JOIN offense_blocking AS pff   
  FULL OUTER JOIN defense_summary AS pff 
  ON fa.Player = pff.player
  WHERE
    Type IN ("UFA", "Void")
--    AND fa.Pos in ("WR", "TE", "HB", "FB", "QB", "LT", "RT", "LG", "RG", "C") 
    AND fa.Pos in ("EDGE", "IDL", "S", "CB", "LB") 
    AND pff.Player IS NULL